In [12]:
import pandas as pd
from pathlib import Path
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import quantstats as qs
import warnings
warnings.simplefilter(action="ignore", category=UserWarning) 
warnings.simplefilter(action="ignore", category=RuntimeWarning)  

import sys

In [ ]:
import pyarrow

print(pd.__version__)
print(pyarrow.__version__)

2.1.4
14.0.2


In [1]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("data")
files = sorted(DATA_DIR.glob("data2_*.parquet"))

data2 = {}
for file in files:
    q = file.stem.replace("data2_", "")
    data2[q] = pd.read_parquet(file)


In [2]:
q, df = next(iter(data2.items()))

print("Kwartał:", q)
print(df.head())
print("\nKolumny:", df.columns)
print("\nIndex:", type(df.index))


Kwartał: 2023_Q1
                   datetime      AUD       CAD  XAG  XAU
0 2023-01-01 17:35:00+00:00      NaN       NaN  NaN  NaN
1 2023-01-01 18:00:00+00:00      NaN       NaN  NaN  NaN
2 2023-01-01 18:05:00+00:00  0.68142       NaN  NaN  NaN
3 2023-01-01 18:15:00+00:00  0.68123  0.739557  NaN  NaN
4 2023-01-01 18:20:00+00:00  0.68083  0.739552  NaN  NaN

Kolumny: Index(['datetime', 'AUD', 'CAD', 'XAG', 'XAU'], dtype='object')

Index: <class 'pandas.core.indexes.range.RangeIndex'>


### porządkowanie danych

In [4]:
def normalize_group2_quarter(df):
    df = df.copy()

    # datetime jako index
    df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
    df = df.set_index("datetime")

    df = df.sort_index()

    return df


In [5]:
data2_norm = {}

for q, df in data2.items():
    data2_norm[q] = normalize_group2_quarter(df)

q, df = next(iter(data2_norm.items()))
print(df.head())
print(type(df.index))


                               AUD       CAD  XAG  XAU
datetime                                              
2023-01-01 17:35:00+00:00      NaN       NaN  NaN  NaN
2023-01-01 18:00:00+00:00      NaN       NaN  NaN  NaN
2023-01-01 18:05:00+00:00  0.68142       NaN  NaN  NaN
2023-01-01 18:15:00+00:00  0.68123  0.739557  NaN  NaN
2023-01-01 18:20:00+00:00  0.68083  0.739552  NaN  NaN
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [6]:
df.index.min(), df.index.max()

(Timestamp('2023-01-01 17:35:00+0000', tz='UTC'),
 Timestamp('2023-03-31 17:00:00+0000', tz='UTC'))

In [7]:
def clean_group2_quarter(df):
    df = df.copy()

    hour = df.index.hour
    weekday = df.index.weekday

  
    is_daily_break = (hour == 16)

    is_saturday = (weekday == 5)

    is_sunday_preopen = (weekday == 6) & (hour < 18)

    is_friday_postclose = (weekday == 4) & (hour >= 16)

    df = df[~(
        is_daily_break |
        is_saturday |
        is_sunday_preopen |
        is_friday_postclose
    )]

    return df



In [8]:
data2_clean = {}

for q, df in data2_norm.items():
    data2_clean[q] = clean_group2_quarter(df)


In [9]:
for q, df in data2_clean.items():
    print(
        q,
        "rows:", len(df),
        "| start:", df.index.min(),
        "| end:", df.index.max()
    )


2023_Q1 rows: 17543 | start: 2023-01-01 18:00:00+00:00 | end: 2023-03-31 15:55:00+00:00
2023_Q3 rows: 17639 | start: 2023-07-02 18:05:00+00:00 | end: 2023-09-29 15:55:00+00:00
2023_Q4 rows: 17606 | start: 2023-10-01 18:00:00+00:00 | end: 2023-12-29 15:55:00+00:00
2024_Q2 rows: 17781 | start: 2024-04-01 00:00:00+00:00 | end: 2024-06-30 23:55:00+00:00
2024_Q4 rows: 17746 | start: 2024-10-01 00:00:00+00:00 | end: 2024-12-31 15:55:00+00:00
2025_Q1 rows: 17301 | start: 2025-01-01 17:05:00+00:00 | end: 2025-03-31 23:55:00+00:00
2025_Q2 rows: 17778 | start: 2025-04-01 00:05:00+00:00 | end: 2025-06-30 23:55:00+00:00


break = 17–18 CET

dane = UTC

17 CET = 16 UTC

### czy opłaca się tradować indywidualnie?

In [10]:
assets = ["AUD", "CAD", "XAU", "XAG"]

def compute_log_returns(df):
    return np.log(df / df.shift(1))

In [13]:
autocorr = {}

for asset in assets:
    autocorr[asset] = {}
    for q, df in data2_clean.items():
        r = compute_log_returns(df)[asset].dropna()
        autocorr[asset][q] = r.autocorr(lag=1)

autocorr_df = pd.DataFrame(autocorr)
autocorr_df


,AUD,CAD,XAU,XAG
2023_Q1,-0.022490,-0.025832,0.003498,-0.046741
2023_Q3,-0.010200,0.011466,-0.008315,0.004776
2023_Q4,-0.008160,-0.024654,0.038023,-0.025845
2024_Q2,-0.003432,-0.015394,-0.017530,-0.014451
2024_Q4,-0.001691,0.050446,0.005425,0.006599
2025_Q1,-0.022896,-0.048633,0.012425,-0.022693
2025_Q2,-0.002505,-0.018331,0.011546,0.023835


Wartości autokorelacji są niewielkie (bliskie zera) – rzędu ±0.01–0.05.

In [14]:
stability = {}

for asset in assets:
    signs = np.sign(autocorr_df[asset])
    stability[asset] = {
        "mean_autocorr": autocorr_df[asset].mean(),
        "std_autocorr": autocorr_df[asset].std(),
        "negative_ratio": (signs < 0).mean(), 
        "positive_ratio": (signs > 0).mean()
    }

pd.DataFrame(stability).T

,mean_autocorr,std_autocorr,negative_ratio,positive_ratio
AUD,-0.010196,0.009072,1.000000,0.000000
CAD,-0.010133,0.032069,0.714286,0.285714
XAU,0.006439,0.017613,0.285714,0.714286
XAG,-0.010646,0.023866,0.571429,0.428571


Tabela stability agreguje informacje o zachowaniu autokorelacji w czasie.
Wszystkie wartości są bardzo bliskie zera, co potwierdza słabą autokorelację

In [15]:
vol = {}

for asset in assets:
    vols = []
    for q, df in data2_clean.items():
        r = compute_log_returns(df)[asset].dropna()
        vols.append(r.std())
    vol[asset] = np.mean(vols)

vol

{'AUD': 0.0004121348787297674,
 'CAD': 0.00024801090720211503,
 'XAU': 0.0005484662397734193,
 'XAG': 0.0010841917495561634}

Najmniej zmienne: CAD

Najbardziej zmienne: XAG (srebro) 

Metale mają wyraźnie wyższą zmienność niż waluty

### Badanie spreadów

In [16]:
spreads = [
    ("XAU", "XAG"),
    ("AUD", "CAD")
]

In [17]:
spread_autocorr = {}

for a, b in spreads:
    name = f"{a}-{b}"
    spread_autocorr[name] = {}
    
    for q, df in data2_clean.items():
        r = compute_log_returns(df)
        spread = (r[a] - r[b]).dropna()
        spread_autocorr[name][q] = spread.autocorr(lag=1)

spread_autocorr_df = pd.DataFrame(spread_autocorr)
spread_autocorr_df


,XAU-XAG,AUD-CAD
2023_Q1,-0.075440,-0.035455
2023_Q3,-0.013635,-0.004675
2023_Q4,-0.032637,-0.012115
2024_Q2,-0.024071,-0.017315
2024_Q4,-0.013498,-0.013438
2025_Q1,-0.049530,-0.067125
2025_Q2,0.007525,-0.024645


XAU–XAG

-Większość kwartałów z ujemną autokorelacją

-Jednorazowy epizod dodatni (2025_Q2) → możliwa zmiana reżimu / szum

AUD–CAD

-Autokorelacja ujemna w każdym kwartale

-Bardzo spójne zachowanie w czasie

In [18]:
spread_stability = {}

for spread in spread_autocorr_df.columns:
    signs = np.sign(spread_autocorr_df[spread])
    spread_stability[spread] = {
        "mean_autocorr": spread_autocorr_df[spread].mean(),
        "std_autocorr": spread_autocorr_df[spread].std(),
        "negative_ratio": (signs < 0).mean(),
        "positive_ratio": (signs > 0).mean()
    }

pd.DataFrame(spread_stability).T


,mean_autocorr,std_autocorr,negative_ratio,positive_ratio
XAU-XAG,-0.028755,0.027159,0.857143,0.142857
AUD-CAD,-0.024967,0.021039,1.000000,0.000000


Średnia autokorelacja jest ~2–3× silniejsza niż dla pojedynczych aktywów

In [19]:
spread_vol = {}

for a, b in spreads:
    name = f"{a}-{b}"
    vols = []
    
    for q, df in data2_clean.items():
        r = compute_log_returns(df)
        spread = (r[a] - r[b]).dropna()
        vols.append(spread.std())
        
    spread_vol[name] = np.mean(vols)

spread_vol


{'XAU-XAG': 0.0008184329831161994, 'AUD-CAD': 0.0003114904856347825}

Spread AUD–CAD jest:

-znacznie stabilniejszy

-łatwiejszy do zarządzania ryzykiem

XAU–XAG:

- ~2.6× wyższa zmienność

- większy potencjał zysku, ale też większe ryzyko

In [20]:
corrs = {}

for a, b in spreads:
    vals = []
    for q, df in data2_clean.items():
        r = compute_log_returns(df)[[a, b]].dropna()
        vals.append(r[a].corr(r[b]))
    corrs[f"{a}-{b}"] = np.mean(vals)

corrs


{'XAU-XAG': 0.6881165237249213, 'AUD-CAD': 0.6666891668316369}

Korelacje są wysokie i stabilne

### Portfel

In [21]:
portfolios = {
    "FX": ["AUD", "CAD"],
    "Metals": ["XAU", "XAG"],
    "All": ["AUD", "CAD", "XAU", "XAG"]
}

In [22]:
portfolio_autocorr = {}

for name, assets_p in portfolios.items():
    portfolio_autocorr[name] = {}
    
    for q, df in data2_clean.items():
        r = compute_log_returns(df)[assets_p]
        port_ret = r.mean(axis=1).dropna()   # równe wagi
        portfolio_autocorr[name][q] = port_ret.autocorr(lag=1)

portfolio_autocorr_df = pd.DataFrame(portfolio_autocorr)
portfolio_autocorr_df

,FX,Metals,All
2023_Q1,-0.019954,-0.024651,-0.022565
2023_Q3,-0.005790,0.008620,0.008323
2023_Q4,-0.012261,-0.009014,-0.006348
2024_Q2,-0.004355,-0.012730,-0.018848
2024_Q4,0.017136,0.011778,0.013241
2025_Q1,-0.028474,-0.005396,-0.009341
2025_Q2,0.000063,0.023379,0.019567


Autokorelacje są małe i bliskie zera, podobnie jak dla pojedynczych aktywów.

Portfel wygładza ekstremalne zachowania składników

In [23]:
portfolio_stability = {}

for p in portfolio_autocorr_df.columns:
    signs = np.sign(portfolio_autocorr_df[p])
    portfolio_stability[p] = {
        "mean_autocorr": portfolio_autocorr_df[p].mean(),
        "std_autocorr": portfolio_autocorr_df[p].std(),
        "negative_ratio": (signs < 0).mean(),
        "positive_ratio": (signs > 0).mean()
    }

pd.DataFrame(portfolio_stability).T


,mean_autocorr,std_autocorr,negative_ratio,positive_ratio
FX,-0.007662,0.014688,0.714286,0.285714
Metals,-0.001145,0.016485,0.571429,0.428571
All,-0.002281,0.016243,0.571429,0.428571


Całkiem losowe

In [24]:
portfolio_vol = {}

for name, assets_p in portfolios.items():
    vols = []
    for q, df in data2_clean.items():
        r = compute_log_returns(df)[assets_p]
        port_ret = r.mean(axis=1).dropna()
        vols.append(port_ret.std())
    portfolio_vol[name] = np.mean(vols)

portfolio_vol


{'FX': 0.00030310083513552757,
 'Metals': 0.0007572025473034684,
 'All': 0.0004660546134840153}

FX:

- Najniższa zmienność

- Najlepsza kontrola ryzyka

Metals:

- Najwyższa zmienność

- Dywersyfikacja słabsza (XAU i XAG silnie skorelowane)

## Najsensowniejszy jest trading spreadów